In [1]:
import os
import random
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score

In [31]:
class MYDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.images = df.iloc[:,5:].values
        self.coef = df.iloc[:,1:5].values
        self.labels = df.iloc[:, 0].values
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        coef = self.coef[idx]
        
        image = torch.tensor(image, dtype=torch.float)
        label = torch.tensor(label, dtype=torch.float)
        coef = torch.tensor(coef, dtype=torch.float)

        return image, coef, label

In [32]:
dataset = pd.read_csv("D://CO2_data1.csv", encoding="utf-8")

In [33]:
def compute_distances(P, C):
    A = (P**2).sum(axis=1, keepdims=True)
 
    B = (C**2).sum(axis=1, keepdims=True).T
 
    return np.sqrt(A + B - 2* np.dot(P, C.T))

In [47]:
train_df = pd.DataFrame()



ycor = dataset.lat
#ycor = dataset.lon
label = dataset.fCO2

train_df['label'] = label

train_df['beta'] = np.ones(623)
train_df['Chl'] = dataset.Chl
train_df['Temp'] = dataset.Temp
train_df['Salt'] = dataset.Salt

alist = dataset.lon
temp = []
for i in alist:
    if i < 0:
        i = i+360
    temp.append(i)
xcor = temp

cor_df = pd.DataFrame()
cor_df['xcor'] = xcor
cor_df['ycor'] = ycor

a = [[110.0, 0.0], [290.0,0.0], [110.0, 70.0], [290.0, 70.0]]
b = np.array(a)

cor_li = cor_df.to_numpy()
dis_li = compute_distances(cor_li, b)
dis_df = pd.DataFrame(dis_li)
train_df = train_df.join(dis_df)



train_data = MYDataset(train_df)
#test_data = MYDataset(test_df)
train_loader = DataLoader(train_data, batch_size=50, shuffle=True, num_workers=0, drop_last=True)
#test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=0)

In [48]:
train_df


,label,beta,Chl,Temp,Salt,0,1,2,3
0,0.640840,1.0,-0.778654,1.087273,0.172380,56.090329,126.554830,77.788977,137.554080
1,0.405739,1.0,-0.808094,1.110043,0.021629,57.316010,125.678658,77.782549,136.235550
2,0.403523,1.0,-0.786881,1.117319,0.126096,56.631484,126.795603,76.368351,136.755713
3,0.225175,1.0,-0.786611,1.085791,-0.077296,57.590147,125.803915,77.081937,135.836759
4,0.311748,1.0,-0.828534,1.142110,0.063110,56.926488,126.927637,75.667860,136.365776
...,...,...,...,...,...,...,...,...,...
618,-0.818805,1.0,-0.679605,0.595628,1.005389,31.593116,161.580089,50.031240,166.171974
619,-1.223775,1.0,-0.550274,0.499140,1.539144,30.579814,164.697070,47.959618,168.790180
620,-1.212453,1.0,-0.621767,0.562813,1.266849,31.154855,163.708964,48.328304,167.826175
621,-1.024903,1.0,-0.629206,0.559075,1.113024,31.750984,162.721004,48.714731,166.862593


In [57]:
class GNNWR(nn.Module):
    def __init__(self, insize, outsize):
        super(GNNWR, self).__init__()
        self.insize = insize
        self.outsize = outsize

        lastsize = self.insize
        thissize = 0
        self.fc = nn.Sequential()
        i = 2

        self.fc.add_module("full"+str(1), nn.Linear(4, 600))
        # self.fc.add_module("batc"+str(1), nn.BatchNorm1d(600))
        # self.fc.add_module("acti"+str(1), nn.PReLU(init=0.4))
        # self.fc.add_module("drop"+str(1), nn.Dropout(0.2))

        lastsize = 600
        while math.pow(2, int(math.log2(lastsize))) >= max(128, outsize + 1):
            if i == 1:
                thissize = int(math.pow(2, int(math.log2(lastsize))))
            else:
                thissize = int(math.pow(2, int(math.log2(lastsize)) - 1))
            
            self.fc.add_module("full"+str(i), nn.Linear(lastsize, thissize))
            self.fc.add_module("batc"+str(i), nn.BatchNorm1d(thissize))
            self.fc.add_module("acti"+str(i), nn.PReLU(init=0.4))
            
            self.fc.add_module("drop"+str(i), nn.Dropout(0.2))

            lastsize = thissize
            i = i + 1

        self.fc.add_module("full"+str(i), nn.Linear(lastsize, outsize))
        
    def forward(self, x):
        x = self.fc(x)
        return x

model = GNNWR(623, 4)
criterion = nn.MSELoss(reduce=True)
optimizer = optim.SGD(model.parameters(), lr=0.1)

d:\Miniconda\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [58]:
r2 = 0
weightlist = []
for i in range(1,2):
    temp = []
    temp.append(2.463e-16)
    temp.append(-0.1239)
    temp.append(0.5236)
    temp.append(0.0319)
    weightlist.append(temp)
out = nn.Linear(4, 1, bias = False)
out.weight = nn.Parameter(torch.tensor(weightlist), requires_grad=False)

def train(epoch):
    model.train()
    train_loss = 0
    global r2
    global out
    for data, coef, label in train_loader:
        data = data.view(50, -1)
        label = label.view(50, -1)
        optimizer.zero_grad()

        output = model(data)
        output = output.mul(coef)
        output = out(output)

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        a = output.view(-1).detach().numpy()
        b = label.view(-1).numpy()
        if epoch % 100 == 0:
            r2 = r2_score(a, b)

        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f} \tR2: {:.6f}'.format(epoch, train_loss, r2))

In [59]:
for epoch in range(1, 200000+1):
    train(epoch)


Epoch: 1 	Training Loss: 0.671032 	R2: 0.000000
Epoch: 2 	Training Loss: 0.564789 	R2: 0.000000
Epoch: 3 	Training Loss: 0.560738 	R2: 0.000000
Epoch: 4 	Training Loss: 0.530218 	R2: 0.000000
Epoch: 5 	Training Loss: 0.518015 	R2: 0.000000
Epoch: 6 	Training Loss: 0.549064 	R2: 0.000000
Epoch: 7 	Training Loss: 0.517901 	R2: 0.000000
Epoch: 8 	Training Loss: 0.510109 	R2: 0.000000
Epoch: 9 	Training Loss: 0.535499 	R2: 0.000000
Epoch: 10 	Training Loss: 0.516492 	R2: 0.000000
Epoch: 11 	Training Loss: 0.517389 	R2: 0.000000
Epoch: 12 	Training Loss: 0.520535 	R2: 0.000000
Epoch: 13 	Training Loss: 0.512175 	R2: 0.000000
Epoch: 14 	Training Loss: 0.520999 	R2: 0.000000
Epoch: 15 	Training Loss: 0.498224 	R2: 0.000000
Epoch: 16 	Training Loss: 0.502381 	R2: 0.000000
Epoch: 17 	Training Loss: 0.493802 	R2: 0.000000
Epoch: 18 	Training Loss: 0.508343 	R2: 0.000000
Epoch: 19 	Training Loss: 0.515987 	R2: 0.000000
Epoch: 20 	Training Loss: 0.500544 	R2: 0.000000
Epoch: 21 	Training Loss: 0.5

KeyboardInterrupt: 